In [2]:
import pandas as pd
import os
import glob
import re
import random
from tqdm.notebook import tqdm
import numpy as np

In [5]:
annotator_name = "tban"
df_all = pd.DataFrame()
for idx, filename in tqdm(enumerate(glob.glob('socialiq_taylor_new_qa_2021_04/*.txt'))):
    video_id = re.match(pattern=r'socialiq_taylor_new_qa_2021_04/(?P<video_id>.+).txt', string=filename).group('video_id')

    previous_problematic_line = None
    
    question = None
    a_list = []
    i_list = []
    with open(filename, 'r') as f: # open in readonly mode
        for cnt, line in enumerate(f):
            if not line.strip():
                continue  # skip empty line
                
            if previous_problematic_line != None:
                line = ''.join([previous_problematic_line.rstrip(), line])


            m = re.match(pattern=r"(?P<line_type>(q|q\d+|a|i)+): (?P<text>.+)", string=line)
            if not m:
                previous_problematic_line = line
                continue
            else:
                previous_problematic_line = None
                
            line_dict = m.groupdict()
            
            if line_dict['line_type'][0] == 'q':
                # append previous question to df
                if question:
                    df_all = df_all.append({'video_id': video_id, 'question': question, 'a_list': a_list, 'i_list': i_list}, ignore_index=True)
                    #print(df_qa_a4)
                question = {'text': line_dict['text'], 'annotator': annotator_name}
                a_list = []
                i_list = []

            elif line_dict['line_type'][0] == 'a':
                answer = {'text': line_dict['text'], 'annotator': annotator_name}
                a_list.append(answer)

            elif line[0] == 'i':
                answer = {'text': line_dict['text'], 'annotator': annotator_name}
                i_list.append(answer)

            else:
                print("Error; Line starts with unknown type!")
                print('{}: Line {}: {}'.format(filename, cnt, line))
                exit(-1)

                    
        # when finished the last line, append the last question
        df_all = df_all.append({'video_id': video_id, 'question': question, 'a_list': a_list, 'i_list': i_list},
                                   ignore_index=True)
            

0it [00:00, ?it/s]

In [6]:
df_all

,a_list,i_list,question,video_id
0,"[{'text': 'He is a child', 'annotator': 'tban'}]","[{'text': 'He is a a male', 'annotator': 'tban'}]",{'text': 'What is unusual about the boy having...,qblGVEjp1cY_
1,"[{'text': 'He was ambitious to start it', 'ann...","[{'text': 'He was bullied into it', 'annotator...",{'text': 'What can be inferred about the boy a...,qblGVEjp1cY_
2,[{'text': 'He already has a plan for his futur...,[{'text': 'He already has a plan for his futur...,{'text': 'What is a sign of the boy's certaint...,qblGVEjp1cY_
3,"[{'text': 'It was out of generosity', 'annotat...","[{'text': 'It was out of guilt', 'annotator': ...",{'text': 'Why did the boy give his employee a ...,qblGVEjp1cY_
4,"[{'text': 'His frusturations', 'annotator': 't...","[{'text': 'His greed', 'annotator': 'tban'}]",{'text': 'What drove the boy to start a busine...,qblGVEjp1cY_
...,...,...,...,...
477,"[{'text': 'Certain', 'annotator': 'tban'}]","[{'text': 'Overwhelmed', 'annotator': 'tban'}]",{'text': 'How does the judge feel about the ca...,wgN6kONTrpc_
478,"[{'text': 'He feels uneasy', 'annotator': 'tba...","[{'text': 'He feels confident', 'annotator': '...","{'text': 'Why does the defendant chuckle?', 'a...",wgN6kONTrpc_
479,"[{'text': 'Amused', 'annotator': 'tban'}]","[{'text': 'Nervous', 'annotator': 'tban'}]",{'text': 'How does the woman accompanying the ...,wgN6kONTrpc_
480,"[{'text': 'To avoid responsibility', 'annotato...","[{'text': 'To avoid questioning', 'annotator':...",{'text': 'Why does the defendant make excuses?...,wgN6kONTrpc_


In [7]:
df_all[df_all.a_list.str.len() ==0]

,a_list,i_list,question,video_id
18,[],[],"{'text': ' ', 'annotator': 'tban'}",OqL_u-uGIi0_
320,[],"[{'text': 'She feels grateful', 'annotator': '...",{'text': 'How does the woman feel after return...,tmkqp3VpDCE_
374,[],[],None,NnhpG0nEC84_


In [8]:
df_all = df_all.drop(df_all[df_all.a_list.str.len() ==0].index)

In [9]:
df_permute = pd.DataFrame()
for index, row in df_all.iterrows():
    video_id = row['video_id']
    question_dict = row['question']
    if question_dict is None:
        import ipdb
        ipdb.set_trace()
    question = question_dict['text']

    question_annotator = question_dict['annotator']
    for a_dict in row['a_list']:
        for i_dict in row['i_list']:
            df_permute = df_permute.append({'video_id': video_id, 'question': question, 'a': a_dict['text'] , 'i': i_dict['text'],
                                            'q_annotator': question_annotator,
                                            'a_annotator': a_dict['annotator'], 'i_annotator': i_dict['annotator']
                                           }, ignore_index=True)
df_permute['qai_id'] = df_permute.index

In [10]:
df_permute

,a,a_annotator,i,i_annotator,q_annotator,question,video_id,qai_id
0,He is a child,tban,He is a a male,tban,tban,What is unusual about the boy having a business?,qblGVEjp1cY_,0
1,He was ambitious to start it,tban,He was bullied into it,tban,tban,What can be inferred about the boy and his bus...,qblGVEjp1cY_,1
2,He already has a plan for his future education,tban,He already has a plan for his future retirement,tban,tban,What is a sign of the boy's certainty about hi...,qblGVEjp1cY_,2
3,It was out of generosity,tban,It was out of guilt,tban,tban,Why did the boy give his employee a pay raise?,qblGVEjp1cY_,3
4,His frusturations,tban,His greed,tban,tban,What drove the boy to start a business?,qblGVEjp1cY_,4
...,...,...,...,...,...,...,...,...
474,Certain,tban,Overwhelmed,tban,tban,How does the judge feel about the case?,wgN6kONTrpc_,474
475,He feels uneasy,tban,He feels confident,tban,tban,Why does the defendant chuckle?,wgN6kONTrpc_,475
476,Amused,tban,Nervous,tban,tban,How does the woman accompanying the defendant ...,wgN6kONTrpc_,476
477,To avoid responsibility,tban,To avoid questioning,tban,tban,Why does the defendant make excuses?,wgN6kONTrpc_,477


In [11]:
df_permute.video_id.unique().shape

(84,)

In [12]:
# dataset too small and all from valid set, so using a random selection instead
84 * 0.7

58.8

In [13]:
# standard_train_fold=['W_GrHtwZez8','3wIejfT9l30','gcDnKQul_c8','qr69jLeQdRA','lo0R1mvjDT8','vKkpvQlHEG8','B6p6X1LSjiA','8m_3eBsy22Y','gfA1xa-BMCg','EqXKrS3gPN4','LTUojzYVUUI','FJF56lmDqQo','QC_4iR0tyvE','E0TBOKN8J2E','g67e0hDT1oQ','s3Czwcz3E-o','D56yCIgqqgk','QQpIBRLlJzo','4pYqIEQow2s','72ltfGTYqpQ','ofvOXABptcY','YlVkBzXo0YM','4VA4kqMnEqA','OWsT2rkqCk8','YIZpD3fEfgs','qh8JWHlIgcE','DW2umNQrQU0','vC63_bemI2I','g8cyMIFcC_g','n6-ef_YHeJU','vqFmKLl2hq4','k5bk1efKBSI','xpXCQkb4LUI','4yr_etbfZtQ','zYeBxnAm2wI','JW2HHfQiGVs','o6DEc3OYQfU','SJSmbF9W9PQ','gKuBUQVcDJM','P8YRuxu8dQU','KBj3TocgvOA','FWBCTZiijEM','lC8nM75AqUk','dU7L1hvMx9Y','Q25kn317a2M','_tBXwqueWcg','ipnGPeRIy2k','97AUfvzQ_1E','WVnNoiQKhPc','pc3__b1uF4Q','ooUw7Hq8YLg','uivjORoSW0k','G3xzem7HSME','CoMz3JOnZFo','cGU1Pepn1hU','yRtPan09Ek0','mNcdlLIOdNw','3udl28R3yIA','-99aZZhUgRk','RgGLnaS7smE','nzj7Wg4DAbs','0SuGC2ifkIA','Iu6_k2ok00U','zzWaf1z9kgk','T1icdxz0TNs','pZCz9-b5Grs','kIfsmKj42XE','EC77tcJZIdU','ixQbCXLUUj8','kYnmc_AVfMs','QtANwxed36M','xWjiz7ZrHkA','bBRWF0wju-c','kmgsC68hIL8','e6ppqFNBkLo','AiIrjf-s128','eKQKEi2-0Ws','FkLblGfWAvY','DClIawJYpHs','lcmI9_aypI0','zT-1nFMmt9Q','kGoON1J872w','C2PneBztZ3g','-QQ0Lv4_--4','1akNUksCpRU','XZn3wQhEXWQ','QQKG6cd-sBI','58DqoE56OWc','xwQBg9pVxrQ','UwPd_D3y3Pk','QUd09jZIQEk','DiaDblUd-lw','jvpj4qdPRE0','QHeNKMxsqYE','XE4ifwZqvEs','tfVvLs379Oo','h9hAaQOanZY','A6Pz9V6LzcU','-nO3dBeh5n0','a80o4DGxt7Q','QqWrqHC8wjs','Bg_tJvCA8zw','4oh_PsdY-W0','GGEXxniRfWQ','VIVkYG31Oas','mJwWxbSR6r0','PnqJALSs6so','vLS1o1F6KqE','_LUX70mXcEE','WLBB9CqfMCk','6qNawyzVGbc','4An2AF2rWGk','hjGsGtihBpc','-VNNzqqnaew','7GRWqlKfgmg','L9U9hvUf42g','EWUfDU8TWn4','B5ltukfhtw8','B2V9PFGQBH4','B-gHVjv4_c4','i3itBKdwE7M','nEpJaJyeRy8','DelU5tQ4grw','Va54WZgPTdY','hkAFdIrTR00','ZeYarFkvoGw','9eqze5JWNjY','h7YTPuEMgaE','G4heS2754l4','8Kv4F0D210A','Bd_vAawM9LA','Cv4Xj4fIkRo','-daGjyKKNio','ON45DvDMSk4','T8JwNZBJ_wI','4W192A7g5KY','nCQB2AaVgOY','spdhjKK6zCo','shL1gWm9qdg','D1Cil5n_-zs','hKfNp8NU82o','9cFEh0aaOOo','eS9U1QO0F7M','Z_FMSWtuU6A','YnqZfVCUSiA','7bc_qfRmPK0','-8GAQpsV4Qo','cq1er8IWz1U','wDgVPV_t0AE','cXTjL-f-msU','lTk5zkc0i8M','Wo1RxRjXyYw','NFKdaj1Qsek','o4CKGkaTn-A','i5YckMkwmm4','ilotZqzaZgU','kztkcj-WAvw','IXbAYg5pp9M','qRM1D4jE09w','2XFVnzr4Vho','gjX78p5tvfo','lKPRa_4hnlE','nblf7Yw4jys','eDqEcrIRxgQ','11BN2alqrFU','uk0Ntd8lJ98','B7Nbbxh3m1Q','caOaW604Tqc','Ilf38Achvzk','LmCJIBsQjOY','X9BE2oUSOXM','HSEi0RXGVq8','tv-hgegVq0k','mN1Z5xEOy10','ykTHGJLCstU','WczPTuyEt5c','HFPGeaEPy9o','2ZkzLAQszHA','tvcrwTcA5iw','HCgv_HNoJrY','SBp5KuQC_lA','0a2lv4IwZFY','br0mu7r-ak0','4HHR_3HJdEQ','uJUP839M2WM','vsNVU7y6dUE','nqY3tv-y62A','GHBKZZuA314','sqDIGuzt38w','EeClqsYITso','HzH0cBmHg5k','lmcuesVvLf0','ZoInHw_M_H4','9QdaNUrq1EQ','5_uSZcXMV7s','t-dmFuH7TyM','hd8bXHCvZME','jiMUoVjQ5uI','Qf2fBuiddNI','qDpGgd4oTQ8','YVHJpAROBvQ','YZR6LEk3doM','_Ice5RkbWUY','DXyaQVlRVkY','fZuk-TaECZo','Xs1sMqpc1oM','1GJqfyzfCWU','2a01Rg2g2Z8','KoH4OKx4x1Y','wRMF9hqdcTI','CggDN9EIuNY','F0wIBTfLnE8','AHXwnFvqYDk','1A7dqFxx8wU','jrv2BW_c8jA','Nra5F1tQQww','-2NF882DbaM','47U9SVOiw4o','VNM7Z7hir_I','x9lr8OomuJ4','SzCgZK1KMio','ecALuiFDRT0','XwUcQU9BFDI','WR0hwJp9AOA','8-Hi9NmF4rM','2c_nei61mQQ','PR0y7IlMEOs','zVrWAxxLuA0','PR04klFMPYQ','yDDRstyk9Dg','XjAmvLVbE3E','0B9VJfzqwhM','DWmUHNpOJxI','p6CRBNmX_gY','R4QbJL80Ri4','HMkOO15nye8','qzCjToACNrU','qXG69NfiePE','zWaV_PTlCxk','ZVR5lhbmGw0','MdG9Lkk8VWo','pK1WAx4jJzE','sQ3pkACdX84','uH5JkfBKD9M','Jp_KHLvQcuw','_s42gOg2WSU','3EwNcTzx-Bs','-sp3Q524oO4','C08WmKiwcSs','bMuoPr5-Yt4','9qK9VQDELpc','bb08nFwfoxA','ap9vRY_Vdwc','rB6WmWvvyxg','Eg307HcpbJE','APshm-9gPgI','Tfw2mq2wJls','4AmVjblOvy4','fpxPstb2DAU','02A2a-aEvmI','i7JGn06gsnA','rq-7zoXQ69Y','RFcuAhpQdXU','dONZkRDs4k4','PjBxYw4SJVg','YTwEFUfX4zA','gbVOyKifrAo','MlteErDn4to','DB7de4nC2rc','5NHofFOpsDU','KHgGasfOFPg','US7hym404bQ','o6hMDs4rBmw','GzPIbX1pzDg','5BLjOCK2SlM','AQ7wbfX_av0','6rkV4QRcVnk','OMkfujDPpwc','badtXoOJaf8','nLu13aVrNcQ','GCZ5aagOddY','JNGcJEZ6rwA','R-xtt7w9Do4','3yovMKR__4Q','n5V7TVYiVas','xNxw4rbwh68','mxa4KXSz9rw','xs8O-MFu4yU','rVWZuXDkc4A','_T3uHSwRgdU','31ktAMJocw8','a6Ke1YThz4o','iFgOoTRflnw','SrMN3rwvCsE','8_Av3cDcoR8','SxtSW5HFkvg','nmWplIQhvoA','fC_Z5HlK9Pw','AciwXaRfh3k','Z86hdiW6l6c','3nUKwvFsjA4','kf9RtsXQPWU','MM0YOB-cSWA','tXG-qPZJj-8','O24d_rJ4uDo','tafBLziHJ0g','hL2u93brqiA','s_y9h4hK2PM','2NE4KCCfutk','P3QEOJWxXI8','QoIsjc4-GIg','H-q0zh-XGVc','MsjnLoTKAXo','ajFmgmUSYAc','teg6qTE9Hjs','B1VB7vVQNQg','ngHnomR-114','N4fvzhsdTio','1j1C3dlwh7E','LcHtLypALog','KB_p2QvvLGw','ukn4Yw4mB1Y','TyFWoohX0bo','luJceOt47UM','3xdQIW24aj0','zHqGiZQTxfw','U4r9ePA6RkQ','2GRzO0p9uVU','Vlh0sMMjU04','zXbortaLKDE','N3QuD26DuCI','b9aeM__20E8','kRh1zXFKC_o','k-kyKv_kKGU','nQqY3j8btbI','XHBZkek8OSU','Bks4JX95dD8','3_-Bjqs6AxA','RZPBl5-cu3c','fhqu7ve9MA4','U_JrsRlieNQ','XYBx__4iAUw','A3WbCRfad-w','xWRNBOXoLf8','iovKlisBCzU','SAgYiERRDPY','DK8s_btC8F8','1CjUHNYzW1E','cOlibbx5sx0','4vPrTC5qMh0','bCWEOlvi5fY','vjZDdaFTubE','h-H1LddWxo8','Y7eYfHIyn3M','sheEL099ADM','L3uDQ0S1Iis','IsgFVkMnqJc','3qp3AeWmt38','7doQf8xjFVg','8y-N6UDxTxQ','d43n4njmxcE','Yzq6fNeXf6w','AZCs9VoHeBo','PsOnuG1EBjQ','n_mTiDeQvWg','RZzaa_PqNJI','29rvfxBQBQA','D0a2KWuL4S0','D1FXpqUivtU','_tYPnaK5gkI','0B7UgDEzcUM','64mWOoj68qo','gAPPzmRb4r0','XYviM5xevC8','yDrkv0iVtsk','Q5YJ7mh8zWc','Zx7c8huezqY','IlLFTI24Qkw','6W77wcXg2no','fHW461eiQp8','9l2W_GDiNyE','HwIRCeUCyxw','XFP-qBKzDBY','ElghrCC2Rbs','tmkqp3VpDCE','jYSuKn09_e4','_oqc_t0mbsQ','n5_HdNzf03Q','FybhAns3or8','8i0Vr6DiBCQ','rZLwPui5TQk','utkUkvYq-zM','GMidefrr1MM','bLVm1vfXRw8','2Wk9JY6ic0k','5RS1CKa3JVg','oY6BfHkNFPY','0_8zVtd5VLY','rnbtRiLamsw','_KE_-EdMhDI','1B_swZA4Hqk','oW3UPfXHSUs','mVnqP-vLpuo','FiLWlqVE9Fg','hRkl5WhbQLc','grTg3dzQDZI','YNmgvI44N5Q','OLw7cIJApMI','3m-54UiEqzg','HN75tPziZAo','jYL4gMsGZgE','2SIhhPzrjVA','hcu4zY2HUQY','2MrFWB__GIA','QRRzvOvG21E','UzzPYlIeSvo','AKAtC7easns','muuW2EU8nrA','0djStqAuc_E','vlnKSMw5v1o','VOHCh8U9XQY','9PJb4cFWOfY','ullXTZ0jjZw','RsqRaVYQ6Cc','DZsBei4nCkU','_xChA_vIuxE','XKyumlBmix8','bT_DEZz99VQ','I0izJOlMJiM','3eCZ8haia58','b0yONlMjxjs','QytntXWJIe8','_0at8kXKWSw','ogIbkhMeJFI','rFJIgAmT8dE','F2Xul-ihUVc','L49M8C4wjVU','srWtQnseRyE','fDe50VbOU44','ZuBzOV1ADkk','deKPBy_uLkg','p9GrpGa8Uys','8SGQ0VdXvAg','tbwPN9fZb2Q','eTnuG394AcY','pXvVnvQOMQ0','STbJ8hJFMAg','4_BudcPRi7E','N188QSyfmeQ','ZKTIOHRjlHM','4HxgOizA2Ow','NZtIGzAzJZM','4iw1jTY-X3A','COYJC6dvB8I','c2pwnHLaYTQ','Kjlt_FgKPgA','-ZOjNsOHtOU','47lUTksozNI','g8D-LyfTrRs','hezHSFSwa08','WQxM_EK5aiM','56EbHWYK_q0','UUuXfyfCaL4','UCW_UH-k-ec','jfKDozXX_Uo','fz0q7YKjp48','LtxWWUBt7ro','GI8LoYEYKI0','oxZYBldwpP4','cuR-l2qCxBc','hnfkZx4jmpA','Me-A3eOhgXs','8xFtIsyRvNE','ory7fcHYXhQ','8fN6D1VOHlo','QFTvmsHgFYM','LBd5x_fe4Jc','x6YzIqb6Cas','8Rk4sGEBJlM','aRQLU3IwNYs','V-qtKSEGAac','2jMvc5VoavE','nGah7qST1dI','q45sJ2n2XPg','1a4Gx6UHdI8','aS01LwpC23g','g7OMgsD7T74','y1Y02_oZP8U','8MK9frCMoWA','2gy8H-wYzic','LTEiibVnRgI','BEOdicifuqM','CY2D1L1JtKU','KWS6OcNbvLA','6V0UfejAo_E','WV1ncOsZjog','_UJNNySGM6Q','aNOuoSVlunM','VP4rHzYyuL0','ZP8ACbJ677I','Sir2QeCh4B0','-jDIwv4wCsU','RJnquSNVuuw','NR9v3PBJw8g','EGK2P1cOJJc','GK4_G33fXFU','RtjJBtFXJaI','wZoXjg1x4Yg','Cn_Mlwouwng','lacVKwbsE7Q','kzhJb5jcH58','f-BbAnnQVtY','TDxlMelzl10','CwanEycyH_8','rmZGPAXE0oA','NysITFb_Wbs','0hCihQ5Kep0','_zuODgCQ6O8','No6mB6V1wL4','yK1RQv3S8KE','sGAj25JPwcg','x6yr8W3mc38','66ojfophGys','9m0d0RaWpfY','zqgqVF6lWB0','F2mIH0vlI9c','ncHMwblapdI','fmuEMg2fh_8','AQX2Q-V2Uh8','LchHXKL6xZY','Mek_AQ5DbUs','8NL5jXoa-Jc','EJdboFptQ3o','6kYu7-5EyU8','jE1tmy-g6tw','s5ak_9z3Cp8','b3I1tK1Iyzc','e4mvg9r6_cI','f3Ch2aIlXWo','s5jrdHASx04','L75hdqt98nw','q0r-dKImVLk','17PtS1Qx8kU','sjzVFt59eds','jP2HuCwfKFA','Fy6BOTB4sXw','urYGhhMOToU','uB3FdWmnFZU','VeV7vvfipA0','4wdeBJ39Cuw','YsBSR_z9BR8','6I7Ktp4dV_s','6AzXxhPKh8U','VsSGubvfPiA','jdrVAQrzt9Y','UJLb8PfQYl8','SsuLW2JpjEA','jKguXsNkJ4w','SQ8aRKG9660','D2VcClclMbs','vTLkSpY_aYg','hBdsfj0YPO8','ASqnnZpsX1M','-0REX0yx4QA','nyqh7vhU3X0','0DBfvvIVmqY','EUIIWsgDpZY','25Qq8k7V83g','jbHKeVsI35M','HDhwReMUBsA','ziwYbVx_-qg','uXlJ3ezSZpg','C6RMS4F6LDc','1iH8ajlHJ1M','BDEUrfqlwcg','BiV9eJU8Gsw','VPTjROKYhlU','jK08J1811uA','kOIWYXDRR7s','WSqbeVe4jXo','afXewnGZXKs','YgfaKTH9RbY','yIhH-_BXPr8','3zjz6ryPvIg','FgnO3muvvoM','WhsZxUWSHaQ','S63inYNCsCM','LD1RAPiT_7A','XFGAQrEUaeU','xdiFzcpmmJc','09j-Mg5guGg','wA4i3eHKsTQ','lB0mkAa1vfM','N-6zVmVuTs0','IsEykad2V9U','IE6r8Pk91T0','U-Y_HkD_0O4','atEkAkPfpUY','mPAESQEQoms','IHU9Jc_NUuk','3d--LpVQxDo','xITR-FIi_oA','ZYzql-Y1sP4','VxS7HHz0mRI','5slZuaphzVs','aai7dDBNXBs','OTdFPlXfFj4','7Oum_c5Seis','o92pxWhZomM','lickge5rPdc','c67D5bP0Hg4','Wg7ppxAgcuw','gImYPbTTZko','KCbXRRvnFj8','w_WAB7TXtXQ','xaQYLAM1LHE','abOuBvUfQk4','-bSM6iswghE','PQW7zw2URes','wz3nVBPVgIA','1YOzwKKbuPo','2YGrrsKs-Xg','xWLmbDdg9tY','Z7MknXjNJSg','O_xHNbx5jqI','VTPz9z7M6B8','2G-B5upjLjM','tVvGairu3Hs','sqIR2N5izHo','Ob3hr2H3VD4','qe--y0apjcU','Nck6BZga7TQ','gDVmHsYgJUA','hRcSU9-krNU','MBDZbACupsc','96YOZOU7ggo','Hv0PyGfxbpw','iwCdv9iR8P8','aDJJBMXiwiI','Obde542xA9I','ON0rtQyNS20','gBs-CkxGXy8','bCKOVlsSluU','G7bkiEh7_AM','zzERcgFrTrA','SjrMprYa608','E4MUXs4IHtY','Y6fVeuHTx0w','awpHn196aVs','NcHQjL_WlxQ','TCCYCSACA2Y','ktdgC1dJkOA','vSQvk9P1dus','kVh1Jw2D9NY','B7XIUxyTi_8','7grGUUPbEbo','lJ83ILGA8yI','0LDAgvbRtk4','rnj_bAv7Hyo','j7cUdCFEpeU','hqzF4IDaIYE','QCR7uyowjhM','0aNFS1cLs4k','dACF-Mz-X8M','N1fVL4AQEW8','dvisqlHIKpM','cHpER0dG1o8','RAUSVB_Nu9k','d89i7OY2yTw','wbmW_Z1oPJw','BC0dD13bwEw','r7rpWLLPQSA','M4blAdS6r3Q','Xbx0Dl90wO0','kmi_liqBsdU','b1OedrPQ464','qow_n-oNEt8','5OUcvUDMMWE','nAkqWGO1T-c','YuU1QV7N9yo','W-V0sdbQA-M','oYM9bFGb8B8','AHiA9hohKr8','kU6YY2z7z0I','fce2RtEvPr8','E2IdU5lgaH4','dVJAvMbb8H4','ZuYTtKZUkJc','epy3Dy2FUOI','-9NhaKWMtWU','4Ry2bE-WRqA','vE0R-Gw_GEA','9jRkACywckE','tNami65DCyE','LiqyOoGW-I8','sty54JE9DHw','A4gVxvYFA3M','gDUFvLWl-Oc','ApExci9PnNM','MIFz86h5nEA','tth90qiKXgY','h35dZhHkuFM','j1CTHVQ8Z3k','ucYhn366lts','ICBNX0i855Q','noHN3H3gWPQ','F2YbeTjcpfs','MHVrwCEWLPI','D2g3gTRkv0U','6xQv6ozrz90','iOdEJMNEzyI','KaIzZrMb2og','DE5S7W8ZfnI','PVD-SzVd_UQ','43fC9xuQRCY','ndsKiHu63oM','cwoR3fkcJ9g','Yz4NWtsQm1w','Nqf15ViHSmQ','5fy7S3jCyAg','ggLOXOiq7WE','T3cwJlY8OYk','1YFLBjR-swo','cQREa5Y-jqk','wYLmgQy6zq8','licUm-aEaCY','l1jW3OMXUzs','yMC03qoBlE0','OT3MVRv0TT4','EwAb8ZW5Eiw','iDgaqD7CWXU','Fy3Mi8rOB3U','0gcuigYJ2cw','2ihOXaU0I8o','m075kb3aV04','VopLr9b_IgM','erOpqmubBL4','KWSDwS4S6Ss','gf5mfpSDFKM','pMGhqE76kQA','TpH0DD3MCUQ','ifxLhziWjm0','LoMhBo8ATBM','7_lpdZhf28E','K25zmgYg5s4','1Za8BtLgKv8','oRBPxefAHTY','PR2fDvbCoAU','DpTB4TDKIa0','YI5l5ltMl8g','zoE6RKgvBlI','13tG38-Ojn4','lUyKpfbB9M8','lkeVfgI0eEk','mr88Ud5V9uE','e6zn4UlO0fU','ztlOyCk5pcg','29vnZjb39u0','N7K5DQvMWXM','bwzH7ceQX8Y','uNU6H62jNxA','90P3VEbzUK0','2DTwXdqmRiM','aXiMaioTUkg','KpljzfIVBDM','fV1o_g6uzuI','A48AJ_5nWsc','1mHjMNZZvFo','MDQTH8WkAvQ','OFia3dWgaoI','xpIFaKnNvco','p6IwwFevo7o','dI5D3aTgjZk','3esHlM0cBx4','X0jMZoxUL2A','cGTFuTIgc88','_AuZO31q62g','ZfZhi8nnnLI','8w41NfRyWqE','nZiIVvEJ8m0','HPszYa77CkM','iBL0FcUTFT8','ALbnaCezgdM','kefqa4xre2I','KBMAUGQpHBU','Nx5VK6DUUEY','FAaWqJLCCd0','O8iOYngEUBU','_Z1tcdf6qkI','WXuRTQPKwqY','NgPP6UXVkYU','q53HUAKB9oU','7FYHA728nBI','qQPl5ySv3Fk','xbhXd8lj2s0','onbBEbC24SQ','sZ9gSPUdZIA','o7Ax6SRTGks','YAYfMkxhRiQ','4HN0caXjW3s','Ot1aZoxt9PU','3eaASCCJB4U','bJ-G8xiLB6o','sJR9QjezRGg','QR43TImr0dQ','YIJ0mZYi1O4','Ham3IQQzoU8','6dCClwMqJK8','13HM_96pUIo','WBOT0Tqpbag','jMZLrPxp31E','dZPwXsbohK4','fL3_AauvjJ4','I4mItsGR3uI','Q3PJuGz0Njs','4tLBy9FGS5A','qhlmvp3VDzI','rMRWJEvKopk','Vwn_QS9vB1g','KvbeKlGeNRU','waE2GdoBW68','UUukBV82P9A','VX671ftXpMU','iiDdvLrSUG4','5WgFDQUjg5s','2K09qUoN1Oo','9L1tM3fOb80','GcImUUGmZ3I','9hn6Z1o-IYI','4KAvDyGzz4E','ajVTImleJlk','alg7qHta0Sk','TewPhK6CZ-Q','4_jXi0nzuow','BH8FUBW4IIE','xXIq7YPkdUQ','-RpZEe4w4fY','MqiOBIxouw4','87yBSfgwoUI','Ate-1815RNA','7wLDCFduiLY','qN-ZCqaSHHk','34XCuNsQ7O8','eS8SpCRASr0','3vjA8sDxDuQ','P5ZOwNK6n9U','ujj3zYdBd0k','Mf76yyTY7Ss','QHGeDg6XX6U','ytEgqN-BdKA','8TDAP0KNIIw','yq1cX1lAyQg','im_uLJKzs-4','iNr9xdc6cVA','pFUXcA1fp6g','ZMJw752P7z8','PfwwCpAy0-0','8ACAI_Z7aLM','imUigBNF-TE','79I7_vkwaeg','OPdbdjctx2I','KzN6XWDEmXI','b-FX9NOVQOM','J94uO-urSTg','Vbp7qtUXQFA','I7GvG0WYfOo','1IHDvzYwqgE','T6x-kDiQsWM','JxbV5wGpXc8','C0g5RjQ7cRE','P3UP62l7g6Q','NTpj7quCIqQ','ctHj7R35dL0','SWNXZbasPvQ','RHRPK2O9R4w','PIqmZBVVH0c','TA3Xy53hMMY','zf2tZ0HIiPQ','T86T9wMn77s','HEke6Dlhqtw','EEDZjwA1wM8','CXmRmrBPDII','jtl5XK7QP38','e8v9i_ksUyY','t4-ZulW5np4','mO8TK-QaIf8','NmWjnYUkT_s']
# standard_valid_fold=['PiYhxJzWLzo','ZyIQkgixS_o','4Vic0qKl64Y','4EZxURAhU6U','yKGfDlxvR_Y','K-bZQJ3P9N0','SstGaYWCNoQ','B6PpxrnttDg','H0Qdz8bSkv0','44MVdpDEQJs','G1wsCworwWk','40mpZRU47T4','3uk6rKXbG1M','5XEQ8rYl1qs','LcMBahfo0NA','pawEXwLjTlo','_yHLgEDXZbI','mA402F5K47o','n8Cn_KhcR7o','MCAH-zHLTLE','NnhpG0nEC84','G4ROcoq32rQ','aSZ_eLxuLAs','ahcAFnY6iAY','bgczomH1kLk','n71IBjXHnrQ','Md4QnipNYqM','00m9ssEAnU4','WI9QnhX7vcI','ks66e-O4YCQ','Csy2RxzkbaM','K6v2QYiMdCE','aw-fKJhcQE4','jh5PklItWjA','8wLCmDtCDAM','u_pE5O64Q9I','6b1QbKtmaZ0','-DTqvzmUw74','egw67gXKK3A','GeZIBgX7vkg','P2rLv-vjSOs','kpMLnFxvi6Y','DuXGDE6tolY','ACPPfJtYCVc','wB_hjqZQ1UY','t7qDtNdJAlk','2nDh8MQuS-Y','uANIooMR9a0','a0tn33wZGVo','F4rSKCXqEw0','xQwrPjLUwmo','igH3ixpts2g','7GRTyxc4uMU','4LGe265pwvU','izCiPuiGe9E','PFA-RmV_wG0','BUumpYIgVg4','j5SKmUoL9Tg','sPxoGNvnVzg','j3pLDghHKyc','IbkyL0pDtEc','CNHBsxOZd80','3oj7mCSydoM','1MwN5nDajWs','G22mJGndp14','460_-P8pK8E','JMzilFvwNXE','tjogri9eYzs','NTySVAtrdQc','iEENyD0JiRE','CbMVjQV9b40','5h-SslT--8E','FaLaPEnjeqY','lTfsqoo-jKg','YRmjALBdTdE','OsQdzsIMFPQ','mpHoYhIFKNI','nRfU_c4QlaQ','KWnV1Aa6VQ8','O951zoCuU7Q','VPoNcAeOycw','S0xTq15pzJU','fsBzpr4k3rY','wf1LTMcVFEc','S8vC5FNRDTU','JRYjFh_hHBs','TTY4WKXoPac','ThpV4osXuCk','GbYGoWvJpwI','ocg0MbBfCS0','PCVvpS7w-2w','2hLcCnZzRtY','EaPaLCuXjT8','O5rTU5EA1C8','wu9D9hdwOQ8','-hnBHBN8p5A','Ddbyb8zVKG0','qAs7UWhTyaE','N5sGxcAJFCA','z6qavchy4Ho','2im0kvBEIrg','aqGNOsZFdBU','bC9hc4cqHGY','lpAMi2lwjo0','geaSpx-R4Kc','pG8zAQGgL1o','Am6NHDbj6XA','FositxHjuUk','TyR3GMJJVhA','dKxXtOyMmYc','hrhX40bQYY0','ihP926ccYDw','-Lp96hoSUC8','JW3OfSCZlhc','otna1VHHCow','eTph1-CG280','JXYyQamYw84','vVaZzurwxeI','RS5x5GW5bEE','vaHpVjqCnNE','T0jM37coZPo','6tAfdCTnToY','OqL_u-uGIi0','Wp0xufdpjqc','SADub7W22Zg','9kLNVTm3Z90']


In [14]:
# train_idx = np.random.choice(df_permute.video_id.unique(), size=59, replace=False)
train_idx = pd.read_json('/work/jianing3/lm-bias/data/socialiq_taylor_new_qa_2021_03/socialiq_permute_train.json').video_id.unique()

train_idx.shape

(59,)

In [15]:
# df_train_dataset = df_permute[df_permute['video_id'].isin(standard_train_fold)].reset_index(drop=True)
# df_valid_dataset = df_permute[~df_permute['video_id'].isin(df_train_dataset.video_id)].reset_index(drop=True)

df_train_dataset = df_permute[df_permute.video_id.isin(train_idx)].reset_index(drop=True)
df_valid_dataset = df_permute[~df_permute['video_id'].isin(df_train_dataset.video_id)].reset_index(drop=True)


In [16]:
df_valid_dataset.video_id.unique().shape

(25,)

In [17]:
df_train_dataset.video_id.unique().shape

(59,)

In [20]:
df_train_dataset.to_json('/work/jianing3/lm-bias/data/socialiq_taylor_new_qa_2021_04/socialiq_permute_train.json')
df_valid_dataset.to_json('/work/jianing3/lm-bias/data/socialiq_taylor_new_qa_2021_04/socialiq_permute_valid.json')

# Extract the original annotations for these videos in the 100 minimal edit annotations (using the qai from same annotators only)

In [123]:
df_train_dataset_original = pd.read_json('/work/jianing3/lm-bias/data/socialiq_permute_same_qai_annotator/socialiq_permute_train.json')
df_valid_dataset_original = pd.read_json('/work/jianing3/lm-bias/data/socialiq_permute_same_qai_annotator/socialiq_permute_valid.json')

In [124]:
df_train_dataset_original_for_minimal_edit = df_train_dataset_original[df_train_dataset_original.video_id.isin(df_permute.video_id.unique())]
df_valid_dataset_original_for_minimal_edit = df_valid_dataset_original[df_valid_dataset_original.video_id.isin(df_permute.video_id.unique())]

In [125]:
df_train_dataset_original_for_minimal_edit.video_id.unique().shape

(70,)

In [126]:
df_valid_dataset_original_for_minimal_edit.video_id.unique().shape

(7,)

In [127]:
df_valid_dataset_original_for_minimal_edit = df_valid_dataset_original_for_minimal_edit.append(df_valid_dataset_original[df_valid_dataset_original.video_id.isin(df_valid_dataset_original.video_id.unique()[:24])])


In [128]:
df_valid_dataset_original_for_minimal_edit.video_id.unique().shape

(30,)

In [129]:
# to keep the comparison fair, only use the first qai triplet for each video

df_train_dataset_original_for_minimal_edit = df_train_dataset_original_for_minimal_edit.drop_duplicates(subset=['video_id', 'question'], keep='first')
df_valid_dataset_original_for_minimal_edit = df_valid_dataset_original_for_minimal_edit.drop_duplicates(subset=['video_id', 'question'], keep='first')
print(df_train_dataset_original_for_minimal_edit.shape)
print(df_valid_dataset_original_for_minimal_edit.shape)

(549, 9)
(282, 9)


In [130]:
df_train_dataset_original_for_minimal_edit.reset_index().to_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit_original_qai_for_comparison/socialiq_permute_train.json')
df_valid_dataset_original_for_minimal_edit.reset_index().to_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit_original_qai_for_comparison/socialiq_permute_valid.json')



# A1 vs. A2 vs. Original spreadsheet generation

In [150]:
df_train_dataset_minimal_edit_annotator1 = pd.read_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit_annotator1/socialiq_permute_train.json')
df_valid_dataset_minimal_edit_annotator1 = pd.read_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit_annotator1/socialiq_permute_valid.json')

df_train_dataset_minimal_edit_annotator2 = pd.read_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit/socialiq_permute_train.json')
df_valid_dataset_minimal_edit_annotator2 = pd.read_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit/socialiq_permute_valid.json')

df_train_dataset_minimal_edit_original = pd.read_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit_original_qai_for_comparison/socialiq_permute_train.json')
df_valid_dataset_minimal_edit_original = pd.read_json('/work/jianing3/lm-bias/data/socialiq_minimal_edit_original_qai_for_comparison/socialiq_permute_valid.json')

In [151]:
df_train_dataset_minimal_edit_annotator1

,a,a_annotator,i,i_annotator,q_annotator,question,video_id,qai_id
0,"Yes, she does what he suggests.",tban,"Yes, she follows him around.",tban,tban,Does the woman seem to trust the man?,XKyumlBmix8,0
1,He wants to seem caring and compassionate.,tban,He wants to seem confident and in control.,tban,tban,Why does the man change his tone of voice when...,XKyumlBmix8,1
2,She is helping the man demonstrate and trusts ...,tban,She is afraid of the man and doesn't trust tha...,tban,tban,Why does the woman stay very still in the begi...,XKyumlBmix8,2
3,The man is a doctor and the woman is his patient,tban,The woman is a doctor and the man is her patient,tban,tban,What is the relationship between the man and t...,XKyumlBmix8,3
4,He is demonstrating the concepts he is talking...,tban,He is holding something and doesn't want it to...,tban,tban,Why is the man interlocking his fingers?,XKyumlBmix8,4
...,...,...,...,...,...,...,...,...
413,They are very excited,tban,They are very bored,tban,tban,How do the children feel in the beginning?,HDhwReMUBsA,590
414,They are working hard.,tban,They are working slowly.,tban,tban,How are the children working?,HDhwReMUBsA,591
415,They are concentrating on what they are doing.,tban,They are trying to ignore the cameras.,tban,tban,Why are the kids looking at their hands and th...,HDhwReMUBsA,592
416,They feel focused.,tban,They feel disappointed.,tban,tban,How do the children feel about their activity?,HDhwReMUBsA,593


In [155]:

df_joined = pd.merge(df_train_dataset_minimal_edit_annotator1, df_train_dataset_minimal_edit_annotator2,  how='left', left_on=['video_id','question'], right_on = ['video_id','question'])
df_joined = pd.merge(df_joined, df_train_dataset_minimal_edit_original,  how='left', left_on=['video_id','question'], right_on = ['video_id','question'])

In [156]:
df_joined

,a_x,a_annotator_x,i_x,i_annotator_x,q_annotator_x,question,video_id,qai_id_x,a_y,a_annotator_y,...,q_annotator_y,qai_id_y,level_0,index,a,a_annotator,i,i_annotator,q_annotator,qai_id
0,"Yes, she does what he suggests.",tban,"Yes, she follows him around.",tban,tban,Does the woman seem to trust the man?,XKyumlBmix8,0,"Yes, she does what he suggests.",tban,...,tban,496.0,10625.0,134196.0,"Yes, she does what he suggests.",nmatis,"No, she seems nervous and apprehensive.",nmatis,nmatis,190996.0
1,He wants to seem caring and compassionate.,tban,He wants to seem confident and in control.,tban,tban,Why does the man change his tone of voice when...,XKyumlBmix8,1,He wants to seem caring and compassionate.,tban,...,tban,497.0,10626.0,134231.0,He wants to seem caring and compassionate.,nmatis,He is afraid of scaring her.,nmatis,nmatis,191031.0
2,She is helping the man demonstrate and trusts ...,tban,She is afraid of the man and doesn't trust tha...,tban,tban,Why does the woman stay very still in the begi...,XKyumlBmix8,2,She is helping the man demonstrate and trusts ...,tban,...,tban,498.0,10629.0,134268.0,She is helping the man demonstrate and trusts ...,eljin,Her spine is very stiff so she cannot move wit...,eljin,eljin,191068.0
3,The man is a doctor and the woman is his patient,tban,The woman is a doctor and the man is her patient,tban,tban,What is the relationship between the man and t...,XKyumlBmix8,3,The man is a doctor and the woman is his patient,tban,...,tban,499.0,10631.0,134304.0,The man is a doctor and the woman is his patient,eljin,The woman is a doctor and the man is his patient,eljin,eljin,191104.0
4,He is demonstrating the concepts he is talking...,tban,He is holding something and doesn't want it to...,tban,tban,Why is the man interlocking his fingers?,XKyumlBmix8,4,He is demonstrating the concepts he is talking...,tban,...,tban,500.0,10633.0,134340.0,He is demonstrating the concepts he is talking...,eljin,"He is thinking, so he interlocks his figners",eljin,eljin,191140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,They are very excited,tban,They are very bored,tban,tban,How do the children feel in the beginning?,HDhwReMUBsA,590,They are very excited,tban,...,tban,37.0,8433.0,106434.0,They are very excited,eljin,They are concentrating very hard,eljin,eljin,149968.0
414,They are working hard.,tban,They are working slowly.,tban,tban,How are the children working?,HDhwReMUBsA,591,They are working hard.,tban,...,tban,38.0,8434.0,106469.0,Individually.,krstokes,Together.,krstokes,krstokes,150003.0
415,They are concentrating on what they are doing.,tban,They are trying to ignore the cameras.,tban,tban,Why are the kids looking at their hands and th...,HDhwReMUBsA,592,They are concentrating on what they are doing.,tban,...,tban,39.0,8446.0,106504.0,They are concentrating on what they are doing.,krstokes,They are scared.,krstokes,krstokes,150038.0
416,They feel focused.,tban,They feel disappointed.,tban,tban,How do the children feel about their activity?,HDhwReMUBsA,593,They feel focused.,tban,...,tban,40.0,8458.0,106539.0,They feel focused.,krstokes,They feel bored.,krstokes,krstokes,150073.0


In [157]:
df_joined.to_csv('/work/jianing3/lm-bias/data/socialiq_minimal_edit_threeway_comparison/socialiq_permute_comparison.csv')

# Extract Amy Li's subset

In [48]:
amy_condition = (df_train_dataset_original.q_annotator == 'Amyl2') & \
                (df_train_dataset_original.a_annotator == 'Amyl2') & \
                (df_train_dataset_original.i_annotator == 'Amyl2')
df_train_dataset_original[amy_condition]

,a,a_annotator,i,i_annotator,q_annotator,question,video_id,qai_id
3426,She feels guilty that she opened the package.,Amyl2,She is angry that she is being scolded.,Amyl2,Amyl2,Why does the little girl pout and look away?,3m-54UiEqzg,3924
3438,She feels thankful and wants to express gratit...,Amyl2,She wants to stop the woman from being upset.,Amyl2,Amyl2,Why does the little girl hug the woman?,3m-54UiEqzg,3936
3450,She is disappointed that her child did not lis...,Amyl2,She is tired.,Amyl2,Amyl2,Why does the woman sigh loudly?,3m-54UiEqzg,3948
13185,He is amused by her reaction to the prank.,Amyl2,He is annoyed by her reaction.,Amyl2,Amyl2,How does the man feel about the prank?,2Wk9JY6ic0k,16887
13209,He knows he is pranking the girl and is not go...,Amyl2,He thinks it is funny that her dinner was ruined.,Amyl2,Amyl2,Why does the man laugh when the girl spits out...,2Wk9JY6ic0k,16911
...,...,...,...,...,...,...,...,...
149128,He wants to patronize and belittle the other man.,Amyl2,He wants to be friendly to the other man.,Amyl2,Amyl2,Why does the shorter man keep smiling at the o...,_s42gOg2WSU,211883
149140,"No, he believes that he feels confident that t...",Amyl2,"Yes, he feels worried that the other man will ...",Amyl2,Amyl2,Does the taller man feel threatened by the sho...,_s42gOg2WSU,211895
149662,He feels thankful that she is helping him talk...,Amyl2,He feels nervous around her.,Amyl2,Amyl2,How does the man feel towards the woman with t...,pMGhqE76kQA,212417
149686,He agrees with what she is saying and taking h...,Amyl2,He is dozing off while she talks.,Amyl2,Amyl2,Why does the man nod when the woman with the s...,pMGhqE76kQA,212441


In [50]:
df_valid_dataset_original[amy_condition].shape

/work/jianing3/anaconda3/envs/pytorch120_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(15, 8)

# Correct answer only (within video swap)

In [3]:
df_train_dataset_original

,a,a_annotator,i,i_annotator,q_annotator,question,video_id,qai_id
0,They are impressed by the revelation,lsack,They are doubtful of the man's words,lsack,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,72
1,They are impressed by the revelation,lsack,The audience is indifferent to what the man in...,hjli,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,73
2,They are impressed by the revelation,lsack,The audience is bored because they already kno...,hjli,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,74
3,They are impressed by the revelation,lsack,The audience is confused by the man in the arm...,asteiner,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,75
4,They are impressed by the revelation,lsack,The audience feels uneasy about the man in the...,asteiner,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,76
...,...,...,...,...,...,...,...,...
156974,To show his interest,seungwom,Because the other man just announced that he i...,gvlastar,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221956
156975,To show his interest,seungwom,Because he wants to impress the other person b...,gvlastar,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221957
156976,To show his interest,seungwom,The man in the black jacket is not saying cong...,seungwom,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221958
156977,To show his interest,seungwom,There is no man in the black jacket in the vid...,seungwom,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221959


In [91]:
random_seed = 2020
random.seed(random_seed)

df_swap_result_train = pd.DataFrame()

for train_video_id in tqdm(df_train_dataset_original.video_id.unique()):
    df_vid = df_train_dataset_original[df_train_dataset_original.video_id == train_video_id]
    
#     print(df_vid.question.unique())
    for question in df_vid.question.unique():
        df_vid_question = df_vid[df_vid.question == question]
        df_vid_not_question = df_vid[df_vid.question != question]
        
        for answer in df_vid_question.a.unique():
            df_vid_question_answer = df_vid_question[df_vid_question.a == answer].iloc[:1]
            df_answer_from_other_question = df_vid_not_question.sample(n=1)  # random sample one incorrect answer from other questions from the same video
#             print(df_vid_question_answer)
#             print(df_answer_from_other_question)
            
            df_swap_result_train = df_swap_result_train.append({
                'a': df_vid_question_answer.a.iloc[0],
                'a_annotator': df_vid_question_answer.a_annotator.iloc[0],
                'i': df_answer_from_other_question.a.iloc[0],
                'i_annotator': df_answer_from_other_question.a_annotator.iloc[0],
                'i_question': df_answer_from_other_question.question.iloc[0],
                'question': df_vid_question_answer.question.iloc[0],
                'q_annotator': df_vid_question_answer.q_annotator.iloc[0],
                'video_id': df_vid_question_answer.video_id.iloc[0],                
            }, ignore_index=True)














  0%|          | 0/887 [00:00<?, ?it/s]











  0%|          | 1/887 [00:00<02:54,  5.07it/s]











  0%|          | 2/887 [00:00<02:45,  5.33it/s]











  0%|          | 3/887 [00:00<02:23,  6.16it/s]











  0%|          | 4/887 [00:00<02:24,  6.11it/s]











  1%|          | 5/887 [00:00<02:25,  6.07it/s]











  1%|          | 6/887 [00:00<02:25,  6.05it/s]











  1%|          | 7/887 [00:01<02:25,  6.06it/s]











  1%|          | 8/887 [00:01<02:53,  5.06it/s]











  1%|          | 9/887 [00:01<02:45,  5.30it/s]











  1%|          | 10/887 [00:01<02:31,  5.78it/s]











  1%|          | 11/887 [00:01<02:12,  6.59it/s]











  1%|▏         | 12/887 [00:01<02:14,  6.52it/s]











  2%|▏         | 14/887 [00:02<02:23,  6.09it/s]











  2%|▏         | 16/887 [00:02<02:04,  7.01it/s]











  2%|▏         | 17/887 [00:02<01:54,  7.58it/s]











  2%|▏         | 18/887 [00:02<01:47,  8.05it/

In [98]:
random_seed = 2020
random.seed(random_seed)

df_swap_result_valid = pd.DataFrame()

for valid_video_id in tqdm(df_valid_dataset_original.video_id.unique()):
    df_vid = df_valid_dataset_original[df_valid_dataset_original.video_id == valid_video_id]
    
#     print(df_vid.question.unique())
    for question in df_vid.question.unique():
        df_vid_question = df_vid[df_vid.question == question]
        df_vid_not_question = df_vid[df_vid.question != question]
        
        for answer in df_vid_question.a.unique():
            df_vid_question_answer = df_vid_question[df_vid_question.a == answer].iloc[:1]
            df_answer_from_other_question = df_vid_not_question.sample(n=1)  # random sample one incorrect answer from other questions from the same video
#             print(df_vid_question_answer)
#             print(df_answer_from_other_question)
            
            df_swap_result_valid = df_swap_result_valid.append({
                'a': df_vid_question_answer.a.iloc[0],
                'a_annotator': df_vid_question_answer.a_annotator.iloc[0],
                'i': df_answer_from_other_question.a.iloc[0],
                'i_annotator': df_answer_from_other_question.a_annotator.iloc[0],
                'i_question': df_answer_from_other_question.question.iloc[0],
                'question': df_vid_question_answer.question.iloc[0],
                'q_annotator': df_vid_question_answer.q_annotator.iloc[0],
                'video_id': df_vid_question_answer.video_id.iloc[0],                
            }, ignore_index=True)

In [101]:
df_swap_result_train['qai_id'] = df_swap_result_train.index
df_swap_result_valid['qai_id'] = df_swap_result_valid.index
df_swap_result_train.to_json('/work/jianing3/lm-bias/data/socialiq_swap/socialiq_permute_train.json')
df_swap_result_valid.to_json('/work/jianing3/lm-bias/data/socialiq_swap/socialiq_permute_valid.json')


In [102]:
df_swap_result_valid

,a,a_annotator,i,i_annotator,i_question,q_annotator,question,video_id,qai_id
0,"The man was being sarcastic, the audience reco...",nturumel@andrew.cmu.edu,"The man is ready with a response, so he has be...",nturumel@andrew.cmu.edu,Has the man been asked this question before?,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,0
1,"No. he was using sarcasm, he did not mean it s...",nturumel@andrew.cmu.edu,The man clearly believes that the topic merits...,nturumel@andrew.cmu.edu,what is the man's (blue jacket) feeling toward...,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,1
2,The audience applaud because they figure out t...,nturumel@andrew.cmu.edu,The man conveys a sense of confidence.,nturumel@andrew.cmu.edu,What done the tone of the man convey?(blue jac...,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,2
3,The sarcasm in the man's statement was funny,nturumel@andrew.cmu.edu,The man lays out a very detailed explanation w...,nturumel@andrew.cmu.edu,Has the man been asked this question before?,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,3
4,The man conveys a sense of confidence.,nturumel@andrew.cmu.edu,"From the context, the host asks him about some...",nturumel@andrew.cmu.edu,Has the man been asked this question before?,nturumel@andrew.cmu.edu,What done the tone of the man convey?(blue jac...,8wLCmDtCDAM,4
...,...,...,...,...,...,...,...,...,...
4615,She is excited.,amui1,"She loves it, and loves working for the woman ...",nmatis,What does the woman in red think about her job?,frankz,When the short-haired woman makes a phone call...,qAs7UWhTyaE,4615
4616,She is surprised.,amui1,She could be or she could not be.,amui1,Is the woman on the left actually nervous as s...,frankz,When the short-haired woman makes a phone call...,qAs7UWhTyaE,4616
4617,She is shocked she is being called by someone ...,skitahar,She is really grateful for her job.,amui1,What does the woman in red think about her job?,frankz,When the short-haired woman makes a phone call...,qAs7UWhTyaE,4617
4618,She did not expect to hear the voice of the wo...,skitahar,She enjoys her job.,skitahar,What does the woman in red think about her job?,frankz,When the short-haired woman makes a phone call...,qAs7UWhTyaE,4618


In [24]:
df_vid_not_question.sample(n=1, random_state=2020)

,a,a_annotator,i,i_annotator,q_annotator,question,video_id,qai_id
84,The man in grey thinks the man in black is sly...,hjli,The man in grey thinks the man in black is not...,asteiner,ccodel,What does the man in grey think about the man ...,g7OMgsD7T74,156


# Same asker and answerer

In [103]:
df_train_dataset_original = pd.read_json('/work/jianing3/lm-bias/data/socialiq_permute_train.json')
df_valid_dataset_original = pd.read_json('/work/jianing3/lm-bias/data/socialiq_permute_valid.json')


In [104]:
df_train_dataset_original

,a,a_annotator,i,i_annotator,q_annotator,question,video_id,qai_id
0,They are impressed by the revelation,lsack,They are doubtful of the man's words,lsack,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,72
1,They are impressed by the revelation,lsack,The audience is indifferent to what the man in...,hjli,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,73
2,They are impressed by the revelation,lsack,The audience is bored because they already kno...,hjli,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,74
3,They are impressed by the revelation,lsack,The audience is confused by the man in the arm...,asteiner,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,75
4,They are impressed by the revelation,lsack,The audience feels uneasy about the man in the...,asteiner,lsack,What does the audience feel about the man in t...,g7OMgsD7T74,76
...,...,...,...,...,...,...,...,...
156974,To show his interest,seungwom,Because the other man just announced that he i...,gvlastar,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221956
156975,To show his interest,seungwom,Because he wants to impress the other person b...,gvlastar,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221957
156976,To show his interest,seungwom,The man in the black jacket is not saying cong...,seungwom,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221958
156977,To show his interest,seungwom,There is no man in the black jacket in the vid...,seungwom,skitahar,Why does the man in the black jacket keep sayi...,SJSmbF9W9PQ,221959


In [107]:
df_train_same_annotator_for_qai = df_train_dataset_original[(df_train_dataset_original.q_annotator == df_train_dataset_original.a_annotator) &
                          (df_train_dataset_original.q_annotator == df_train_dataset_original.i_annotator)]
df_valid_same_annotator_for_qai = df_valid_dataset_original[(df_valid_dataset_original.q_annotator == df_valid_dataset_original.a_annotator) &
                          (df_valid_dataset_original.q_annotator == df_valid_dataset_original.i_annotator)]

In [111]:
df_train_same_annotator_for_qai.reset_index().to_json('/work/jianing3/lm-bias/data/socialiq_permute_same_qai_annotator/socialiq_permute_train.json')
df_valid_same_annotator_for_qai.reset_index().to_json('/work/jianing3/lm-bias/data/socialiq_permute_same_qai_annotator/socialiq_permute_valid.json')

In [112]:
df_valid_same_annotator_for_qai

,a,a_annotator,i,i_annotator,q_annotator,question,video_id,qai_id
0,"The man was being sarcastic, the audience reco...",nturumel@andrew.cmu.edu,The audience believe the man,nturumel@andrew.cmu.edu,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,282
1,"The man was being sarcastic, the audience reco...",nturumel@andrew.cmu.edu,"Yes, The man was telling a joke and that made ...",nturumel@andrew.cmu.edu,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,283
2,"The man was being sarcastic, the audience reco...",nturumel@andrew.cmu.edu,"Yes, The audience found the man's remarks true...",nturumel@andrew.cmu.edu,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,284
3,"No. he was using sarcasm, he did not mean it s...",nturumel@andrew.cmu.edu,The audience believe the man,nturumel@andrew.cmu.edu,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,285
4,"No. he was using sarcasm, he did not mean it s...",nturumel@andrew.cmu.edu,"Yes, The man was telling a joke and that made ...",nturumel@andrew.cmu.edu,nturumel@andrew.cmu.edu,Why do the audience applaud at the man's sugge...,8wLCmDtCDAM,286
...,...,...,...,...,...,...,...,...
19157,"She loves it, and loves working for the woman ...",nmatis,She doesn't like it and thinks it's boring.,nmatis,nmatis,What does the woman in red think about her job?,qAs7UWhTyaE,221996
19192,"Yes, she fidgets in her seat",frankz,"No, she only said it for effect",frankz,frankz,Is the woman on the left actually nervous as s...,qAs7UWhTyaE,222031
19227,She loves her personality.,nmatis,She thinks she's really funny.,nmatis,nmatis,What does the woman in black think about the w...,qAs7UWhTyaE,222066
19262,She looks up to her and is very grateful to her,frankz,She dislikes her,frankz,frankz,How does the woman on the left feel about the ...,qAs7UWhTyaE,222101
